### Starting your first 3Di simulation

In this notebook we will go through the steps of connecting to the threedi-api and
start our first simple 3Di simulation.

Let's first import all required modules for the notebook:

In [ ]:
from datetime import datetime
from getpass import getpass
from openapi_client import ApiException, SimulationsApi, OrganisationsApi
from openapi_client.api import AuthApi
from openapi_client.api import ThreedimodelsApi
from threedi_api_client.threedi_api_client import ThreediApiClient

Then we'll provide our credentials to connect to the threedi-api

In [ ]:
API_HOST = "https://api.3di.live/v3.0"
USERNAME = input("Username")
PASSWORD = getpass("Password")
config = {
    "API_HOST": API_HOST,
    "API_USERNAME": USERNAME,
    "API_PASSWORD": PASSWORD
}

api_client = ThreediApiClient(config=config)

Check if we can connect to the threedi-api with the provided credentials:

In [ ]:
auth_api = AuthApi(api_client)

try:
    user = auth_api.auth_profile_list()
except ApiException as e:
    print("Oops, something went wrong. Maybe you made a typo?")
else:
    print(f"Successfully logged in as {user.username}!")

To run a simulation we'll need a threedi-model. Let's see which threedi-models are
available:

In [ ]:
models_api = ThreedimodelsApi(api_client)

models = models_api.threedimodels_list(limit=5)  # limit to the first 5 results
for model in models.results:
    print(f"{model.name}")

If we already know the name of a specific model, we can also look it up with the following api call:

In [ ]:
my_model = models_api.threedimodels_list(name__icontains='v2_bergermeer')
print(my_model)

my_model = my_model.results[0]

Now that we have a model we are almost ready to create the simulation. But first we'll need 
to get an organisation under which's name we will run the simulation.

Let's see which organisations are available:

In [ ]:
organisation_api = OrganisationsApi(api_client)

organisations = organisation_api.organisations_list()
for organisation in organisations.results:
    print(f"{organisation.name}: {organisation.unique_id}")

# Here I use the following organisation, you should select from the list below
organisation_uuid = "b08433fa47c1401eb9cbd4156034c679"

Now we got all the pieces to create a simulation

In [ ]:
simulation_api = SimulationsApi(api_client)

my_simulation = simulation_api.simulations_create(
    data={
        "name": "my first simulation",
        "threedimodel": my_model.id,
        "organisation": organisation_uuid,
        "start_datetime": datetime.now(),
        "duration": 3600  # in seconds, so we simulate for 1 hour
    }
)

The simulation has now been created but is not yet running. We can see the status of
the simulation with the following call:

In [ ]:
status = simulation_api.simulations_status_list(my_simulation.id)
print(status)

To run this simulation we have to tell it to start:

In [ ]:
simulation_api.simulations_actions_create(my_simulation.id, data={"name": "start"})

Eventually the simulation will finish. You can periodically check its progress by calling
for the status again:

In [ ]:
simulation_api.simulations_status_list(my_simulation.id)


Eventually you should see something like this::

    {
        'created': datetime.datetime(2020, 7, 27, 14, 7, 6, 654905, tzinfo=tzutc()),
        'id': 15866,
        'name': 'finished',
        'paused': False,
        'time': 3600.0
     }

Congratulations, you just made your first simulation via the threedi-api-client!